In [8]:
# Importation des bibliothèques nécessaires
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score
import numpy as np

# Chargement du modèle sauvegardé
checkpoint = torch.load('../results/graph/best_model.pt')

# Initialisation des modèles (encoder et predictor)
encoder.load_state_dict(checkpoint['encoder'])
predictor.load_state_dict(checkpoint['predictor'])

# Déplacement des modèles sur le bon périphérique (CPU ou GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
predictor.to(device)

# Test final sur l'ensemble de test
@torch.no_grad()
def test(data_split):
    encoder.eval()
    predictor.eval()

    # Encoder les données de test
    z = encoder(data_split.x, data_split.edge_index.to(device))

    # Prédiction des arêtes sur l'ensemble de test
    pred = predictor(z, data_split.edge_label_index.to(device))

    # Conversion des résultats en numpy pour calcul des métriques
    pred_cpu = pred.cpu().numpy()
    label_cpu = data_split.edge_label.cpu().numpy()

    # Calcul de l'AUC (Area Under the Curve)
    auc = roc_auc_score(label_cpu, pred_cpu)

    # Calcul de l'accuracy (prédictions binaires)
    pred_binary = (pred > 0.5).cpu().numpy()
    acc = (pred_binary == label_cpu).mean()

    return auc, acc

# Évaluation sur l'ensemble de test
test_auc, test_acc = test(test_data)

# Affichage des résultats de l'évaluation
print(f"Test AUC: {test_auc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Calcul des embeddings finaux
encoder.eval()
with torch.no_grad():
    final_embeddings = encoder(test_data.x.to(device), test_data.edge_index.to(device))

# Sauvegarde des résultats (embeddings, model_state, etc.)
MODEL_OUTPUT = 'final_model_output.pt'
torch.save({
    'embeddings': final_embeddings.cpu(),
    'node_to_idx': node_to_idx,
    'model_state': {
        'encoder': encoder.state_dict(),
        'predictor': predictor.state_dict()
    },
    'metrics': {
        'test_auc': test_auc,
        'test_acc': test_acc
    }
}, MODEL_OUTPUT)

print(f"✅ Modèle sauvegardé dans {MODEL_OUTPUT}")

# Visualisation des embeddings finaux avec t-SNE
embeddings_2d = TSNE(n_components=2, random_state=42).fit_transform(final_embeddings.cpu().numpy())

# Création du graphique
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=test_data.community.cpu().numpy(), cmap='jet', s=5)
plt.colorbar()
plt.title('Visualisation des Embeddings (t-SNE)')
plt.show()



NameError: name 'encoder' is not defined

In [2]:
class GCNEncoder(nn.Module):
    """Encoder GCN qui génère les embeddings"""
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

class EdgePredictor(nn.Module):
    """Predictor pour la prédiction de liens"""
    def __init__(self, in_channels):
        super().__init__()
        self.lin1 = nn.Linear(in_channels * 2, 128)
        self.lin2 = nn.Linear(128, 1)
    
    def forward(self, z, edge_index):
        row, col = edge_index
        z_combined = torch.cat([z[row], z[col]], dim=1)
        x = F.relu(self.lin1(z_combined))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.lin2(x).squeeze()


In [4]:
MODEL_PATH = "../results/graph/best_model.pt"
print(f"\n🔍 Détection automatique des dimensions depuis : {MODEL_PATH}")

# Charger le checkpoint pour inspecter les dimensions
checkpoint = torch.load(MODEL_PATH, map_location='cpu')

# Extraire les dimensions depuis les poids de l'encoder
# conv1.lin.weight a la forme [hidden_channels, in_channels]
# conv2.lin.weight a la forme [out_channels, hidden_channels]
first_conv_weight = checkpoint['encoder']['conv1.lin.weight']
second_conv_weight = checkpoint['encoder']['conv2.lin.weight']

hidden_channels = first_conv_weight.shape[0]
in_channels = first_conv_weight.shape[1]
out_channels = second_conv_weight.shape[0]

print("\n📋 Dimensions détectées automatiquement:")
print(f"  in_channels (features): {in_channels}")
print(f"  hidden_channels (couche cachée): {hidden_channels}")
print(f"  out_channels (embeddings): {out_channels}")
print("="*60)


🔍 Détection automatique des dimensions depuis : ../results/graph/best_model.pt


KeyError: 'conv1.lin.weight'